In [5]:
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.utilities import PythonREPL
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# Prompt
template = """Write Python code to solve the user's problem.
Return only Python code in Markdown format:
````python
...
```"""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

# Model
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=pipe)
repl = PythonREPL()

def extract_code(text: str) -> str:
    """Extract Python code from Markdown code blocks."""
    if "```python" in text:
        return text.split("```python")[1].split("```")[0].strip()
    if "```" in text:
        return text.split("```")[1].split("```")[0].strip()
    return text.strip()

def run_code(code: str) -> str:
    """Execute code in REPL."""
    return repl.run(code)

# Chain
chain = prompt | llm | StrOutputParser() | extract_code | run_code

result = chain.invoke({"input": "Write a program to print a binary tree in an inorder traversal."})
print(result)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


KeyboardInterrupt: 

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

# Setup
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
repl = PythonREPL()

# Prompt
template = """Write Python code to solve the user's problem.
Return only Python code in Markdown format:
````python
...
```"""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

def extract_code(text: str) -> str:
    if "```python" in text:
        return text.split("```python")[1].split("```")[0].strip()
    if "```" in text:
        return text.split("```")[1].split("```")[0].strip()
    return text.strip()

# Split into two chains
code_chain = prompt | llm | StrOutputParser() | extract_code
exec_chain = code_chain | repl.run

# Get code
code = code_chain.invoke({"input": "Write a program to print a binary tree in an inorder traversal."})
print("=== Generated Code ===")
print(code)

# Execute
print("\n=== Output ===")
result = repl.run(code)
print(result)

=== Generated Code ===
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def inorder_traversal(root):
    if root is not None:
        inorder_traversal(root.left)
        print(root.value, end=' ')
        inorder_traversal(root.right)

# Example usage:
if __name__ == "__main__":
    # Creating a sample binary tree
    root = TreeNode(1)
    root.left = TreeNode(2)
    root.right = TreeNode(3)
    root.left.left = TreeNode(4)
    root.left.right = TreeNode(5)

    print("Inorder Traversal of the binary tree:")
    inorder_traversal(root)

=== Output ===

